# Project in Mahine Learning

Loading packages

In [1]:
from utility import *
from scipy.signal import savgol_filter

mySeed = randint(0, 2**32 - 1)
print(mySeed)

1496316948


The Seed to use to compare all different models is 3496236729.

In [2]:
try :
    X1, X2, Y1 = open_data("C:\\Users\\Louis Lovat\\Desktop\\UNIF\\MachineLearning\\Project\\data")
except :
    X1, X2, Y1 = open_data("C:\\Users\\hchri\\Jupyter\\Machine learning\\project_data")
    
X2.drop(["title", "is_adult", "img_url", "description", "Unnamed: 0"], axis=1, inplace=True)
X1_p = X1.drop(["title", "is_adult", "img_url", "description", "Unnamed: 0"], axis=1)
colmns = ['ratings','n_votes','production_year','runtime','release_year']

# General preprocessing

X1_p = Binarize(X1_p, 'studio', 10)
X1_p = Binarize(X1_p, 'genres', 5)

X1_p = strToFloatArray(X1_p, "text_embeddings")
X1_p = strToFloatArray(X1_p, "img_embeddings")

mySeed = 3496236729
otherSeed = 3943876323
n_model_folds = 10
n_parameters_folds = 10

"""
UltimeglobalProcess = Process(X1_p, Y1)
UltimeglobalProcess.setTrainTest()
globalProcess = Process(UltimeglobalProcess.X_train, UltimeglobalProcess.Y_train)
"""

globalProcess = Process(X1_p, Y1)
KF = KFold(n_splits=n_model_folds, shuffle=True, random_state=mySeed)
modelFolds = []; modelTrainIndices = []; modelTestIndices = []
for i, (train_index, test_index) in enumerate(KF.split(globalProcess.X)):
    modelFolds.append(i)
    modelTrainIndices.append(train_index)
    modelTestIndices.append(test_index)


In [3]:
modelName = "linear"
modelResults = np.zeros(n_model_folds)
for i, train_index, test_index in zip(modelFolds, modelTrainIndices, modelTestIndices):
    
    # The purpose of this loop is to compute the Cross-Validation score of the linear model based on the RMSE
    # The score of the different models will be used to choose which model is the best suited for our project
    # Each model will perform in the best conditions (the best parameters are selected for the model)

    # Setting up the train and test sets

    toPrint = f"Running Fold {i} for model Selection : {modelName}"
    print("="*len(toPrint) + "\n" +  toPrint + "\n" + "="*len(toPrint))
    globalProcess.setTrainTest(train_index = train_index, test_index = test_index)

    # Preprocessing

    # Replacing missing values by the mean
    globalProcess.X_train.fillna(globalProcess.X_train["runtime"].mean(), inplace=True)
    globalProcess.X_test.fillna(globalProcess.X_train["runtime"].mean(), inplace=True)

    # Selecting the most relevant components of the embeddings with PCA
    globalProcess.PCA_embeddings("text_embeddings", 100)
    globalProcess.PCA_embeddings("img_embeddings", 100)

    # Removing duplicated data from the training set
    globalProcess.removeDuplicate()
    
    # Keeping only the features which are correlated with the target 
    # Correlation threshold = 0.1
    globalProcess.corrThreshold()

    globalProcess.addModel(modelName)
    


    pred = globalProcess.useModel(modelName)
    pred2 = globalProcess.useModel(modelName, X = globalProcess.X_train)
    modelResults[i] = compute_rmse(pred, globalProcess.Y_test*1e-6)
    #modelResults2 = compute_rmse(pred2, globalProcess.Y_train*1e-6)
    print(f"In these conditions RMSE value on this iteration is : {modelResults[i]}\n")
    #print(f"In these conditions RMSE_train value on this iteration is : {modelResults2}\n")

meanModelResult = np.mean(modelResults)
print(f"The mean RMSE for the {modelName} model in best conditions is {meanModelResult}")





Running Fold 0 for model Selection : linear
In these conditions RMSE value on this iteration is : 37.17164767873171

Running Fold 1 for model Selection : linear
In these conditions RMSE value on this iteration is : 63.59579150393704

Running Fold 2 for model Selection : linear
In these conditions RMSE value on this iteration is : 43.72869436849594

Running Fold 3 for model Selection : linear
In these conditions RMSE value on this iteration is : 72.44890271220495

Running Fold 4 for model Selection : linear
In these conditions RMSE value on this iteration is : 71.83151352957144

Running Fold 5 for model Selection : linear
In these conditions RMSE value on this iteration is : 66.5118911382455

Running Fold 6 for model Selection : linear
In these conditions RMSE value on this iteration is : 51.7491624246578

Running Fold 7 for model Selection : linear
In these conditions RMSE value on this iteration is : 57.79298418756229

Running Fold 8 for model Selection : linear
In these conditions RM

In [9]:
modelName = "linear"

# Preprocessing

# Replacing missing values by the mean
UltimeglobalProcess.X_train.fillna(UltimeglobalProcess.X_train["runtime"].mean(), inplace=True)
UltimeglobalProcess.X_test.fillna(UltimeglobalProcess.X_train["runtime"].mean(), inplace=True)

# Selecting the most relevant components of the embeddings with PCA
UltimeglobalProcess.PCA_embeddings("text_embeddings", 100)
UltimeglobalProcess.PCA_embeddings("img_embeddings", 100)

# Removing duplicated data from the training set
UltimeglobalProcess.removeDuplicate()

# Keeping only the features which are correlated with the target 
# Correlation threshold = 0.1
UltimeglobalProcess.corrThreshold()

UltimeglobalProcess.addModel(modelName)
pred = UltimeglobalProcess.useModel(modelName)
print(compute_rmse(pred, UltimeglobalProcess.Y_test*1e-6))



59.088850535245705


In [4]:
allParameterSets = [(i, p, 'uniform') for i in range(1, 31) for p in range(1,6)]

In [5]:
modelName = "knn"
# allParameterSets = [] # shape (n_combinations, n_parameters_per_set), one set is [n_neighbors, p, weights]
modelResults = np.zeros(n_model_folds)
for i, train_index, test_index in zip(modelFolds, modelTrainIndices, modelTestIndices):
    
    # The purpose of this loop is to compute the Cross-Validation score of the knn model based on the RMSE
    # The score of the different models will be used to choose which model is the best suited for our project
    # Each model will perform in the best conditions (the best parameters are selected for the model)

    # Setting up the train and test sets

    toPrint = f"Running Fold {i} for model Selection : {modelName}"
    print("="*len(toPrint) + "\n" +  toPrint + "\n" + "="*len(toPrint))
    globalProcess.setTrainTest(train_index = train_index, test_index = test_index)

    # Splitting the training data into 10 K-Folds in order to select the best parameters

    subProcess = Process(globalProcess.X_train, globalProcess.Y_train)
    subKF = KFold(n_splits=n_parameters_folds, shuffle=True)

    # Selection of the best parameters

    allParametersResults = np.zeros((len(allParameterSets), n_parameters_folds))
    for j, (sub_train_index, sub_test_index) in enumerate(subKF.split(subProcess.X)):

        # The purpose of this loop is to select the best parameters for the knn model
        # We compute the Cross-Validation score based on the RMSE for each set of parameters
        # The set with the lowest CV-score (mean RMSE on all folds) is chosen as the best set
         
        print(f"\tRunning Fold {j} for parameters' Selection : {modelName}")

        # Setting up the train and test sets

        subProcess.setTrainTest(train_index = sub_train_index, test_index = sub_test_index)

        # Preprocessing

        # Replacing missing values by the mean
        subProcess.X_train.fillna(subProcess.X_train["runtime"].mean(), inplace=True)
        subProcess.X_test.fillna(subProcess.X_train["runtime"].mean(), inplace=True)

        # Selecting the most relevant components of the embeddings with PCA
        subProcess.PCA_embeddings("text_embeddings", 100)
        subProcess.PCA_embeddings("img_embeddings", 100)

        # Removing duplicated data from the training set
        subProcess.removeDuplicate()

        # Removing redundant features because of their correlation
        subProcess.removeRedundantFeatures(threshold = 0.8)

        # We scale the data with minmax (we take as min/max the maximal non-outliers values)
        subProcess.minmaxize(withOutliers = False)

        # Keeping only the features which are correlated with the target 
        # Correlation threshold = 0.1
        subProcess.corrThreshold()

        # Variations of the set of parameters

        for index, parameterSet in zip(range(len(allParameterSets)), allParameterSets):

            # Building and training the model

            subProcess.addModel(modelName, n_neighbors = parameterSet[0], p = parameterSet[1], weights = parameterSet[2])

            pred = subProcess.useModel(modelName)
            # Computing RMSE

            allParametersResults[index, j] = compute_rmse(pred, subProcess.Y_test*1e-6)

    # Computing wich set of parameters is the best one

    meanResults = np.mean(allParametersResults, axis = 1)
    bestParams = allParameterSets[np.argmin(meanResults)]
    print(f"\nBest parameters' values are : {bestParams}")

    # Estimation of the RMSE with optimal parameters

    # Preprocessing

    # Replacing missing values by the mean
    globalProcess.X_train.fillna(globalProcess.X_train["runtime"].mean(), inplace=True)
    globalProcess.X_test.fillna(globalProcess.X_train["runtime"].mean(), inplace=True)

    # Selecting the most relevant components of the embeddings with PCA
    globalProcess.PCA_embeddings("text_embeddings", 100)
    globalProcess.PCA_embeddings("img_embeddings", 100)

    # Removing duplicated data from the training set
    globalProcess.removeDuplicate()

    # Removing redundant features because of their correlation
    globalProcess.removeRedundantFeatures(threshold = 0.8)

    # We scale the data with minmax (we take as min/max the maximal non-outliers values)
    globalProcess.minmaxize(withOutliers = False)

    # Keeping only the features which are correlated with the target 
    # Correlation threshold = 0.1
    globalProcess.corrThreshold()

    globalProcess.addModel(modelName,n_neighbors = bestParams[0], p = bestParams[1], weights = bestParams[2])
    pred = globalProcess.useModel(modelName)
    modelResults[i] = compute_rmse(pred, globalProcess.Y_test*1e-6)
    print(f"In these conditions RMSE value on this iteration is : {modelResults[i]}\n")

meanModelResult = np.mean(modelResults)
print(f"The mean RMSE for the {modelName} model in best conditions is {meanModelResult}")



Running Fold 0 for model Selection : knn
	Running Fold 0 for parameters' Selection : knn
	Running Fold 1 for parameters' Selection : knn
	Running Fold 2 for parameters' Selection : knn
	Running Fold 3 for parameters' Selection : knn
	Running Fold 4 for parameters' Selection : knn
	Running Fold 5 for parameters' Selection : knn
	Running Fold 6 for parameters' Selection : knn
	Running Fold 7 for parameters' Selection : knn
	Running Fold 8 for parameters' Selection : knn
	Running Fold 9 for parameters' Selection : knn

Best parameters' values are : (14, 1, 'uniform')
In these conditions RMSE value on this iteration is : 35.277969640625386

Running Fold 1 for model Selection : knn
	Running Fold 0 for parameters' Selection : knn
	Running Fold 1 for parameters' Selection : knn
	Running Fold 2 for parameters' Selection : knn
	Running Fold 3 for parameters' Selection : knn
	Running Fold 4 for parameters' Selection : knn
	Running Fold 5 for parameters' Selection : knn
	Running Fold 6 for parame

In [ ]:
modelName = "randomForest"
allParameterSets = [] # shape (n_combinations, n_parameters_per_set), one set is [n_estimators, max_depth]
modelResults = np.zeros(n_model_folds)
for i, train_index, test_index in zip(modelFolds, modelTrainIndices, modelTestIndices):
    
    # The purpose of this loop is to compute the Cross-Validation score of the randomForest model based on the RMSE
    # The score of the different models will be used to choose which model is the best suited for our project
    # Each model will perform in the best conditions (the best parameters are selected for the model)

    # Setting up the train and test sets

    toPrint = f"Running Fold {i} for model Selection : {modelName}"
    print("="*len(toPrint) + "\n" +  toPrint + "\n" + "="*len(toPrint))
    globalProcess.setTrainTest(train_index = train_index, test_index = test_index)

    # Splitting the training data into 10 K-Folds in order to select the best parameters

    subProcess = Process(globalProcess.X_train, globalProcess.Y_train)
    subKF = KFold(n_splits=n_parameters_folds, shuffle=True)

    # Selection of the best parameters

    allParametersResults = np.zeros((len(allParameterSets), n_parameters_folds))
    for j, (sub_train_index, sub_test_index) in enumerate(subKF.split(subProcess.X)):

        # The purpose of this loop is to select the best parameters for the randomForest model
        # We compute the Cross-Validation score based on the RMSE for each set of parameters
        # The set with the lowest CV-score (mean RMSE on all folds) is chosen as the best set
         
        print(f"\tRunning Fold {j} for parameters' Selection : {modelName}")

        # Setting up the train and test sets

        subProcess.setTrainTest(train_index = sub_train_index, test_index = sub_test_index)

        # Preprocessing

        # Replacing missing values by the mean
        subProcess.X_train.fillna(subProcess.X_train["runtime"].mean(), inplace=True)
        subProcess.X_test.fillna(subProcess.X_train["runtime"].mean(), inplace=True)

        # Selecting the most relevant components of the embeddings with PCA
        subProcess.PCA_embeddings("text_embeddings", 100)
        subProcess.PCA_embeddings("img_embeddings", 100)

        # Removing duplicated data from the training set
        subProcess.removeDuplicate()

        # Removing redundant features because of their correlation
        subProcess.removeRedundantFeatures(threshold = 0.8)

        # We scale the data with minmax (we take as min/max the maximal non-outliers values)
        subProcess.minmaxize(withOutliers = False)

        # Keeping only the features which are correlated with the target 
        # Correlation threshold = 0.1
        subProcess.corrThreshold()

        # Variations of the set of parameters

        for index, parameterSet in zip(range(len(allParameterSets)), allParameterSets):

            # Building and training the model

            subProcess.addModel(modelName)
            pred = subProcess.useModel(modelName, n_estimators = parameterSet[0], max_depth = parameterSet[1])

            # Computing RMSE

            allParametersResults[index, j] = compute_rmse(pred, subProcess.Y_test*1e-6)

    # Computing wich set of parameters is the best one

    meanResults = np.mean(allParametersResults, axis = 1)
    bestParams = allParameterSets[np.argmin(meanResults)]
    print(f"\nBest parameters' values are : {bestParams}")

    # Estimation of the RMSE with optimal parameters

    # Preprocessing

    # Replacing missing values by the mean
    globalProcess.X_train.fillna(globalProcess.X_train["runtime"].mean(), inplace=True)
    globalProcess.X_test.fillna(globalProcess.X_train["runtime"].mean(), inplace=True)

    # Selecting the most relevant components of the embeddings with PCA
    globalProcess.PCA_embeddings("text_embeddings", 100)
    globalProcess.PCA_embeddings("img_embeddings", 100)

    # Removing duplicated data from the training set
    globalProcess.removeDuplicate()

    # Removing redundant features because of their correlation
    globalProcess.removeRedundantFeatures(threshold = 0.8)

    # We scale the data with minmax (we take as min/max the maximal non-outliers values)
    globalProcess.minmaxize(withOutliers = False)

    # Keeping only the features which are correlated with the target 
    # Correlation threshold = 0.1
    globalProcess.corrThreshold()

    globalProcess.addModel(modelName)
    pred = globalProcess.useModel(modelName, n_estimators = bestParams[0], max_depth = bestParams[1])
    modelResults[i] = compute_rmse(pred, globalProcess.Y_test*1e-6)
    print(f"In these conditions RMSE value on this iteration is : {modelResults[i]}\n")

meanModelResult = np.mean(modelResults)
print(f"The mean RMSE for the {modelName} model in best conditions is {meanModelResult}")



In [4]:
allParameterSets = [([100 * j]*i, 'relu') for i in range(2,6) for j in range(1, 4)]

In [ ]:
modelName = "mlp"
#allParameterSets = [] # shape (n_combinations, n_parameters_per_set), one set is [hidden_layer_sizes, activation]
modelResults = np.zeros(n_model_folds)
for i, train_index, test_index in zip(modelFolds, modelTrainIndices, modelTestIndices):
    
    # The purpose of this loop is to compute the Cross-Validation score of the mlp model based on the RMSE
    # The score of the different models will be used to choose which model is the best suited for our project
    # Each model will perform in the best conditions (the best parameters are selected for the model)

    # Setting up the train and test sets

    toPrint = f"Running Fold {i} for model Selection : {modelName}"
    print("="*len(toPrint) + "\n" +  toPrint + "\n" + "="*len(toPrint))
    globalProcess.setTrainTest(train_index = train_index, test_index = test_index)

    # Splitting the training data into 10 K-Folds in order to select the best parameters

    subProcess = Process(globalProcess.X_train, globalProcess.Y_train)
    subKF = KFold(n_splits=n_parameters_folds, shuffle=True)

    # Selection of the best parameters

    allParametersResults = np.zeros((len(allParameterSets), n_parameters_folds))
    for j, (sub_train_index, sub_test_index) in enumerate(subKF.split(subProcess.X)):

        # The purpose of this loop is to select the best parameters for the mlp model
        # We compute the Cross-Validation score based on the RMSE for each set of parameters
        # The set with the lowest CV-score (mean RMSE on all folds) is chosen as the best set
         
        print(f"\tRunning Fold {j} for parameters' Selection : {modelName}")

        # Setting up the train and test sets

        subProcess.setTrainTest(train_index = sub_train_index, test_index = sub_test_index)

        # Preprocessing

        # Replacing missing values by the mean
        subProcess.X_train.fillna(subProcess.X_train["runtime"].mean(), inplace=True)
        subProcess.X_test.fillna(subProcess.X_train["runtime"].mean(), inplace=True)

        # Selecting the most relevant components of the embeddings with PCA
        subProcess.PCA_embeddings("text_embeddings", 100)
        subProcess.PCA_embeddings("img_embeddings", 100)

        # Removing duplicated data from the training set
        subProcess.removeDuplicate()

        # Removing redundant features because of their correlation
        subProcess.removeRedundantFeatures(threshold = 0.8)

        # We scale the data with minmax (we take as min/max the maximal non-outliers values)
        subProcess.minmaxize(withOutliers = False)

        # Keeping only the features which are correlated with the target 
        # Correlation threshold = 0.1
        subProcess.corrThreshold()

        # Variations of the set of parameters

        for index, parameterSet in zip(range(len(allParameterSets)), allParameterSets):

            # Building and training the model

            subProcess.addModel(modelName, hidden_layer_sizes = parameterSet[0], activation = parameterSet[1], max_iter = 1000)
            pred = subProcess.useModel(modelName)

            # Computing RMSE

            allParametersResults[index, j] = compute_rmse(pred, subProcess.Y_test*1e-6)

    # Computing wich set of parameters is the best one

    meanResults = np.mean(allParametersResults, axis = 1)
    bestParams = allParameterSets[np.argmin(meanResults)]
    print(f"\nBest parameters' values are : {bestParams}")

    # Estimation of the RMSE with optimal parameters

    # Preprocessing

    # Replacing missing values by the mean
    globalProcess.X_train.fillna(globalProcess.X_train["runtime"].mean(), inplace=True)
    globalProcess.X_test.fillna(globalProcess.X_train["runtime"].mean(), inplace=True)

    # Selecting the most relevant components of the embeddings with PCA
    globalProcess.PCA_embeddings("text_embeddings", 100)
    globalProcess.PCA_embeddings("img_embeddings", 100)

    # Removing duplicated data from the training set
    globalProcess.removeDuplicate()

    # Removing redundant features because of their correlation
    globalProcess.removeRedundantFeatures(threshold = 0.8)

    # We scale the data with minmax (we take as min/max the maximal non-outliers values)
    globalProcess.minmaxize(withOutliers = False)

    # Keeping only the features which are correlated with the target 
    # Correlation threshold = 0.1
    globalProcess.corrThreshold()

    globalProcess.addModel(modelName, hidden_layer_sizes = bestParams[0], activation = bestParams[1], max_iter = 1000)
    pred = globalProcess.useModel(modelName)
    modelResults[i] = compute_rmse(pred, globalProcess.Y_test*1e-6)
    print(f"In these conditions RMSE value on this iteration is : {modelResults[i]}\n")

meanModelResult = np.mean(modelResults)
print(f"The mean RMSE for the {modelName} model in best conditions is {meanModelResult}")



Running Fold 0 for model Selection : mlp
	Running Fold 0 for parameters' Selection : mlp
	Running Fold 1 for parameters' Selection : mlp
	Running Fold 2 for parameters' Selection : mlp


C:\Users\Louis Lovat\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [ ]:
print("test")